In [60]:
import streamlit as st
import gpxpy
import folium
from streamlit_folium import st_folium
import pandas as pd
import boto3
import json
import random



def parse_gpx(gpxdf, number):
    points = []
    print(gpxdf)
    gpx_data = gpx = gpxpy.parse(gpxdf['activity_gpx']) 
    color = gpxdf['color']
    filename = gpxdf['filename']
    activity_name = gpxdf['activity_name']
    # print(color)
    for track in gpx_data.tracks:
        for segment in track.segments:
            for point in segment.points:
                points.append({'Latitude': point.latitude, 'Longitude': point.longitude, 'Color': color, 'Number' : number, 'activity_name': activity_name, 'filename': filename})
    return points

def bucket_query_namefiles():
    activity_filename = []
    session = boto3.Session()
    s3 = session.resource('s3') 
    bucket = s3.Bucket('solvesdgs')
    for obj in bucket.objects.all():
        if 'activityfiles' in obj.key:
            activity_filename.append(obj.key)
    return activity_filename[1:]

def generate_random_color():
    # Generate a random hexadecimal color code
    color_code = "#{:06x}".format(random.randint(0, 0xFFFFFF))
    return color_code

def download_files():
    session = boto3.Session()
    s3 = session.resource('s3') 
    files_name = bucket_query_namefiles()
    json_df = pd.DataFrame()
    color_random = generate_random_color()
    for obj_name in files_name:
        content_object = s3.Object('solvesdgs', obj_name)
        file_content = content_object.get()['Body'].read()    
        json_content = json.loads(file_content)
        json_content['filename'] = obj_name
        json_content['color'] = color_random
        content_df = pd.DataFrame([json_content],  index=[0])
        content_df= content_df   #[['activity_gpx','filename', 'color' , 'activity_name']]
        json_df = pd.concat([json_df,content_df], ignore_index=True)
    return json_df

def read_files():
    df_content = download_files()
    # Parse each GPX file
    all_points = []    # for file, color in colors.items():
    for index, row in df_content.iterrows():
        points = parse_gpx(row, index)
        all_points.extend(points)

    return all_points

def plot_map(points):
    m = folium.Map(location=[points[0]['Latitude'], points[0]['Longitude']], zoom_start=12)
    seen = {}
    
    for point in points:
        if not seen.get(point['Number'], False):
            folium.Marker(location=[point['Latitude'], point['Longitude']],  popup= point['activity_name'],
                          icon=folium.Icon(color=point['Color'])).add_to(m)
        seen[point['Number']] = True

    grouped = pd.DataFrame(points).groupby('Number')

    for number_id, group_data in grouped:
        color_1 = group_data['Color'].iloc[0]
        locations = [(row['Latitude'], row['Longitude']) for index, row in group_data.iterrows()]
        folium.PolyLine(locations=locations, color=color_1).add_to(m)
        
    return m

In [61]:
download_files()

,activity_name,activity_description,activity_date,activity_schedule,activity_time,activity_location,activity_capacity,activity_images,activity_link,activity_tweet,activity_age_group,message4users,activity_gpx,activity_comments,filename,color
0,cool hike,Lets hike in otaranta its cool,2024-03-23,6.00am,00:30,Otaranta 8,1,"images/c8a343f185352af293d19a84e9ea1c09.JPG,im...",https://www.google.com/maps,cool hike in the night,13-15,join me lol,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<gpx x...","[{'comment_title': 'xd', 'comment_description'...",activityfiles/128fe47b961aaa66ee39de43b0aed636...,#bc5392
1,otaranta hike,hike in otaranta,2024-03-23,6.00am,00:30,otaranta 8,23,"images/c8a343f185352af293d19a84e9ea1c09.JPG,im...",,,10-12,join lol,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<gpx x...","[{'comment_title': 'Cool shit', 'comment_descr...",activityfiles/54ef005ead386218b081b4a48ec2e6a2...,#bc5392
2,jmt hike,Lets hike in jmt close to aalto,2024-03-23,6.00am,00:30,Jämeräntaival 11,19,"images/3ee5c0e8b06a4d2df65ad441cdb87eb7.JPG,",https://www.geeksforgeeks.org/md5-hash-python/,jmt hike lol,81+,join me lol,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<gpx x...","[{'comment_title': 'asdcfv', 'comment_descript...",activityfiles/bdae5aa40e89e9e4a19955a0d6d78f9d...,#bc5392
3,otaranta hike 2.0,join me on an otaranta hike,2024-03-23,6.00am,00:30,Otaranta 8,18,"images/700c295be4ebc4179e1a82f73dea3dc5.jpg,im...",https://www.geeksforgeeks.org/md5-hash-python/,otaranta hike lol,81+,join me please :(,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<gpx x...","[{'comment_title': 'adadsadad', 'comment_descr...",activityfiles/e0bfd7885866154f629dee224b364704...,#bc5392
